# The Wealth of Cities
## Predicting the Wealth of a City from Satellite Imagery

## Introduction

Accurate measurements of the economic characteristics of cities critically influence research and government policy (Jean, Luo). Such measurements can shape decisions by governments on how to allocate resources and provide infrastructure to improve human livelihoods in a wide-range of situations. Although economic data is readily available for most developed and some developing nations, many regions of the modern world remain unexposed to the benefits of economic analysis because of a lack key measures of economic development and efficiency. Regions such as parts of Africa do not have systems in place to conduct economic surveys or other means of collecting data on their financial situations. In our project we attempt to address this problem by using publicly available satellite images to predict the wealth of a city (or, more generally, a geographic region) based on fundamental features identified in these images and running them through a support vector machine(SVM). Not only would this method be applicable to regions that lack economic data, but could also be applied to cities with a wealth of economic information on a macro level but a dearth on a micro level. For example, cities in America, despite having lots of economic data on state and county levels, could benefit from understanding more granular information in order to improve policy decisions for infrastructure and public support. 


## Related Work

Night-Time Light Data: A Good Proxy Measure for Economic Activity?
Charlotta Mellander, International Business School, Jönköping University, Sweden
José Lobo, School of Human Evolution and Social Change, Arizona State University, USA
Kevin Stolarick, Inclusive Design Research Centre, OCAD University, Canada
Zara Matheson, Former Martin Prosperity Institute, Rotman School of Management, University of Toronto, Canada
http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0139779

Authors of this paper investigated how economic metrics like density, urbanisation, economic growth are related to Night Time Lighting obtained from Satellite Images. They used a "fine grained geo-coded residential and industrial full sample micro-data set for Sweden" and then used correlation analysis and geographically weighted regressions to match it with radiance and saturated light emissions. Interestingly they found that "correlation between NTL and economic activity is strong enough to make it a relatively good proxy for population and establishment density, but the correlation is weaker in relation to wages." Further, they said that they find a "stronger relation between light and density values, than with light and total values. We also find a closer connection between radiance light and economic activity, than with saturated light." 


Finding Poverty in Satellite Images 
Neal Jean, Stanford University, California, USA
Rachel Luo, Stanford University, California, USA
http://cs229.stanford.edu/proj2015/162_report.pdf

This paper uses only day time satellite images because the authors discovered that NTL is almost completely absent in very poor and rural areas, which makes it impossible to gauge economic activity of these regions with night time images. In this study, the authors took the output of a convolutional neural network (CNN), a 4,096-element feature vector, and use these image features along with known survey data from certain parts of Uganda and Tanzania to perform linear regression on continuous wealth measures. Next, they use this model to predict both consumption-based and asset-based wealth measures, and they find that their predictions do in fact "approach survey accuracy."


We take inspiration from both these works and try to extract features and use machine learning techniques that are most feasible for the nature and scale for our project. 


## Features

In order for this approach to work, we need to be able to extract relevant features from the images in order to train our machine learning model. Our model will not be able to predict the wealth of individual houses (i.e., families), but will work on clusters of houses (i.e., neighborhoods and cities) because of the complexity of wealth measurements and tendency for neighborhood to be at a nearly homogeneous economic level. As a result, we will need to extract "cluster" features to process with our SVM.

Features of satellite images that we extract to elucidate the wealth of a region are:
1. Night Time Lighting
2. Percentage of Special Area types
3. Number of Cars

### Night Time Lighting 

This feature has been widely researched and turns out to be a relatively good indicator of economic activity of a region. Many studies have been done to assets to what extent does Night Time Lighting, and its sub-features like radiance and saturation, relate to economic activity and population density. Mellander et al. in their paper "Night-Time Light Data: A Good Proxy Measure for Economic Activity?" say that "We find that the correlation between NTL and economic activity is strong enough to make it a relatively good proxy for population and establishment density, but the correlation is weaker in relation to wages." In this project we wish to use NTL in tandem with our other features to see whether we can predict average wages of a area more accurately. 

We use high resolution Night Images taken by Earth Observatory NASA in 2012. The collection, called Earth at Night 2012, can be found here: http://earthobservatory.nasa.gov/Features/NightLights/page3.php . These are a collection of 9 images: one map-like image of the whole earth of resolution 54000x27000 and 8 regional tiles of resolution 13500x13500. 

To calculate the approximate the light of a location, given its latitude and longitude: 
1. We find it pixel location on the map-like image, 
2. Calculate which tile image that location corresponds to, 
3. Transform the location to the the correct pixel location in the tile image which has higher resolution, 
4. Average the light over the neighboring pixels.

![Locating Seattle on Night Satellite Image](night_time_example.png)

### Percentage of Special Area Types + Roads

When looking at high and low incomes during the day, it becomes apparent that high income areas tend to have amenities that are not available in low income areas.  These include well paved roads, parks, museums and other cultural centers.  So, it is natural to try to predict the average income of a city based on how developed the city seems to be, and how many attractions there are for residents to take part in.  The Google Static Maps API enable us to extract multiple area types from an image of a city.  These include natural land, man-made land, roads, water, areas of interest (museums and cultural centers), and parks.  We created features from this by taking the percentage of land in each city that belonged to each category.  
 
### Number of Cars

Another proxy for economic activity is the number of cars on the roads. Yes, some cities that are poorer than others will have more cars probably due to higher population density, but certainly cities that few to no cars will be the poorest. In order to extract the number of cars on the satellite-view Google Maps image we use a two-fold method: 
1. Template Matching 
2. Variance in the color of the roads

Template matching is a technique in digital image processing for finding small parts of an image which match a template image. In our case, the template image is that of a car, and we use Canny edge detection to match it to our satellite image with the help of the openCV package (cv2). We take the edges of both the road pictures and the template picture, and compute the high similarity points (using the TM_COEFF_NORMED values), and rotate the template image around to account for different road directions.  Calling cv2.matchtemplate returns a grayscale image of correlation values according to the above function.  From these images (one for every rotation we do), we pick out values that are above a threshold and use those to be the number of car "pixels" in the image.  We then take the percentage of car pixels to be one of the features in the SVM.

We also used the satellite-view to find the variance in the color of the roads. This metric has multiple uses for us.  The foremost is that it gives us an indication of the quality of roads in a city, as well paved roads tend to be uniform, and poorly maintained roads will have dirt and grass growing.  It also hints at the number of cars on the road because more the number of cars implies the presence of varied colors in the region of the road, so we wanted to see what impact this feature would have on the overall predictions. Since we already know exactly where the roads are, from the road-map, we can easily find the variance in the color of the pixels corresponding the to the region of the roads, which would give us the required feature. 

Both these features in tandem would give us a good handle on the condition of automotive travel in different cities.  While we thought this would be a good predictor for American cities, where car travel is popular, we were also concerned that in European cities that are incredibly wealthy yet don't have many car owners, this may be a false herring for the SVM.

![Car Edge Template](car_template.png)
![Car Example](car_example.png)


## Training

Finally we use a trained Support Vector Machine to classify new data.  We obtained our training data from http://www.city-data.com/ , which features highly detailed data profiles of all cities in the United States. We used Scikit-learn's SVR to train and predict on our features.  After testing all of the kernels we found that the RBF and sigmoid kernels gave us almost constant predictions for every city, so they were not useful. Of the  poly and linear kernels, the linear one gave us the best fit in the middle range, and the poly kernel was able to predict well for outliers, such as Zurich Switzerland (which has the highest average income in the world).  Ultimately we decided that the linear kernel would be the best for us.

In [147]:
from wealth_predictor import WealthPredictor

# City labels just for reference
training_places = ["San Fransisco, CA", "DC", "Blackwater, AZ", "Sneedsville, TN", "Newtonn, MA", "Mamou, LA"]
training_coords = [((37.710978, -122.500087), (37.802606, -122.404110)),
          ((38.819597, -77.160145), (38.983113, -76.912953)),
          ((33.000521, -111.664691), (33.045859, -111.568733)),
          ((36.513680, -83.234828), (36.555887, -83.166506)),
         ((42.288678, -71.267065), (42.366094, -71.160635)), 
         ((30.624943, -92.425908), (30.644181, -92.410887))]
training_labels = [63024, 57291, 9491, 13719, 102796, 19172]

print "Training WealthPredictor..."
wp = WealthPredictor(15)
wp.train(training_coords, training_labels)
print "Done!"

# Again, city labels just for reference
training_places = ["University Park, NM", "Pittsburgh, PA", "Tenafly, NJ"]
test_coords = [((32.263894, -106.765491), (32.285883, -106.739656)),
               ((40.417268, -80.036749), (40.418268, -80.035749)),
               ((40.901511, -73.982347), (40.936339, -73.930420))]
test_labels = [5520, 28176, 73846]

print "Predicting test coordinates..."
print wp.predict(test_coords)
print "Done!"

Training WealthPredictor...
Done!
Predicting test coordinates...
[ 38231.5  38231.5  38231.5]
Done!


## Results

## Improvement